In [1]:
%matplotlib qt5
# %matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ipywidgets import widgets
from IPython.display import display, clear_output, Javascript
import mne
from mne.io import read_raw_fif
from mne.preprocessing import read_ica
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
import numpy as np
import getpass
import os
# nbconvert related imports
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

import json  # noqa
import pprint  # noqa
params = json.load(open("params.json"))

pprint.pprint({'experiment parameters': params["general"]})
subject_ids = params["general"]["subject_ids"]  
session_ids = params["general"]["session_ids"] 
main_path = params["general"]["data_path"] 

{'experiment parameters': {'NJOBS': 1,
                           'data_path': '/home/pasca/Science/workshop/PracticalMEEG/ds000117/derivatives/meg_derivatives',
                           'data_type': 'fif',
                           'session_ids': ['01', '02'],
                           'short': False,
                           'subject_ids': ['sub-01'],
                           'subjects_dir': '/home/pasca/Science/workshop/PracticalMEEG/ds000117/FSF'}}


## Choose subject ID:

In [3]:
name_sel = widgets.Select(
    description='Subject ID:',
    options=subject_ids
)
display(name_sel)

cond_sel = widgets.RadioButtons(
    description='Condition:',
    options=session_ids,
)
display(cond_sel)

Select(description='Subject ID:', options=('sub-01',), value='sub-01')

RadioButtons(description='Condition:', options=('01', '02'), value='01')

In [4]:
%%capture
if cond_sel.value == session_ids[0]:
    session = session_ids[0]
elif cond_sel.value == session_ids[1]:
    session = session_ids[1]
subj_ID = name_sel.value

In [5]:
preproc_workflow_name = 'preprocessing_dsamp_workflow'
preproc_pipeline_name = 'preproc_meg_dsamp_pipeline'

In [6]:
print('*** sbj ID ->  {} ***'.format(subj_ID))
print('*** session -> {} ***'.format(session))

*** sbj ID ->  sub-01 ***
*** session -> 01 ***


In [7]:
# define file names
pipeline_path = os.path.join(main_path, preproc_workflow_name)
sbj_data_path = os.path.join(main_path, subj_ID)

basename = subj_ID + '_ses-meg_task-facerecognition_run-' + session + '_proc-sss_meg_filt_dsamp'
results_folder = os.path.join(preproc_pipeline_name, '_session_id_' + session + '_subject_id_' + subj_ID)

raw_fname = basename + '.fif'
ica_fname = basename + '_ica.fif'
ica_TS_fname = basename + '_ica-tseries.fif'
report_fname = basename + '-report.html'
ica_solution_fname = basename + '_ica_solution.fif'

raw_file = os.path.join(pipeline_path, results_folder, 'preproc', raw_fname)  # filtered data
raw_ica_file = os.path.join(pipeline_path, results_folder, 'ica', ica_fname)  # cleaned data by MNE
ica_TS_file = os.path.join(pipeline_path, results_folder, 'ica', ica_TS_fname)
ica_solution_file = os.path.join(pipeline_path, results_folder, 'ica', ica_solution_fname)
report_file = os.path.join(pipeline_path, results_folder, 'ica', report_fname)


# Load data

In [8]:
# print('Load raw file -> {}'.format(raw_ica_file))
# raw = read_raw_fif(raw_ica_file, preload=True)
print('Load raw file -> {} \n\n'.format(raw_file))
raw = read_raw_fif(raw_file, preload=True)
ica = read_ica(ica_solution_file)
ica.labels_ = dict()
ica_TS = ica.get_sources(raw)

Load raw file -> /home/pasca/Science/workshop/PracticalMEEG/ds000117/derivatives/meg_derivatives/preprocessing_dsamp_workflow/preproc_meg_dsamp_pipeline/_session_id_01_subject_id_sub-01/preproc/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg_filt_dsamp.fif 


Opening raw data file /home/pasca/Science/workshop/PracticalMEEG/ds000117/derivatives/meg_derivatives/preprocessing_dsamp_workflow/preproc_meg_dsamp_pipeline/_session_id_01_subject_id_sub-01/preproc/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg_filt_dsamp.fif...
    Range : 67800 ... 157799 =    226.000 ...   525.997 secs
Ready.
Reading 0 ... 89999  =      0.000 ...   299.997 secs...
Reading /home/pasca/Science/workshop/PracticalMEEG/ds000117/derivatives/meg_derivatives/preprocessing_dsamp_workflow/preproc_meg_dsamp_pipeline/_session_id_01_subject_id_sub-01/ica/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg_filt_dsamp_ica_solution.fif ...
Now restoring ICA solution ...
Ready.


## Cell below opens an html report in a web-browser

In [9]:
%%bash -s "$report_file"
firefox -new-window $1

In [10]:
ica.exclude
print(raw.info)

<Info | 23 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: []
 ch_names: MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, MEG0132, ...
 chs: 204 Gradiometers, 102 Magnetometers, 74 EEG, 3 Stimulus, 12 misc, 9 CHPI
 custom_ref_applied: False
 description: (meg) Vectorview system at Cambridge
 dev_head_t: MEG device -> head transform
 dig: 137 items (3 Cardinal, 5 HPI, 75 EEG, 54 Extra)
 events: 1 item (list)
 experimenter: MEG
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 hpi_subsystem: 2 items (dict)
 line_freq: 50.0
 lowpass: 40.0 Hz
 meas_date: 1941-03-22 11:04:14 UTC
 meas_id: 4 items (dict)
 nchan: 404
 proc_history: 1 item (list)
 proj_id: 1 item (ndarray)
 proj_name: dgw_studies
 projs: []
 sfreq: 300.0 Hz
 subject_info: 2 items (dict)
>


In [11]:
ica.plot_sources(raw)

Creating RawArray with float64 data, n_channels=68, n_times=90000
    Range : 67800 ... 157799 =    226.000 ...   525.997 secs
Ready.
Using qt as 2D backend.


In [13]:
ica.plot_components(inst=raw)

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x496 with 8 Axes>]

In [14]:
print(ica.exclude)
if ica.exclude:
    ica.plot_properties(raw, picks=ica.exclude)

[25, 30, 27]
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated


## Exclude ICA components

To exclude/include an ICA component **click on mne_browse window**: the **red** ones will be excluded. To keep the new excluded ICA components CLOSE the mne_browe window!

## Apply ica solution to raw data and save the result

Check in the next cells that you are excluding the components you want!!!

In [29]:
# ica.exclude = [1]
print('You want to exclude the following components: ***  {} ***'.format(ica.exclude))
# print(raw.info)
# raw.plot()

You want to exclude the following components: ***  [24, 14, 29] ***


In [31]:
%%capture
ica.apply(raw)
raw.save(raw_ica_file, overwrite=True)
ica.save(ica_solution_file, overwrite=True)

In [32]:
print('You REMOVED the following components: ***  {} ***'.format(ica.exclude))
print('You SAVED the new CLEANED file here: ***  {} ***'.format(raw_ica_file))

You REMOVED the following components: ***  [24, 14, 29] ***
You SAVED the new CLEANED file here: ***  /home/pasca/Science/workshop/PracticalMEEG/ds000117/derivatives/meg_derivatives/preprocessing_dsamp_workflow/preproc_meg_dsamp_pipeline/_session_id_01_subject_id_sub-01/ica/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg_filt_dsamp_ica.fif ***


# From here code to play with the data

In [33]:
## if you want to check some particular component
i = 2
for f in [1, 100]:
    plt.figure()
    ax = plt.axes()
    ica_TS.plot_psd(tmax=60, picks=[i], fmax=f, ax=ax, show=False)
    ax.set_title('IC #' + str(i))
    plt.show()
ica.plot_properties(raw, picks=i)


NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 0.853 (s)
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 0.853 (s)
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated


[<Figure size 700x600 with 6 Axes>]